In [73]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [74]:
faults = pd.read_csv('../data/J1939Faults.csv', low_memory=False)
fault_codes = pd.read_excel('../data/Service Fault Codes.xlsx')
diagnostic_data = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')
pd.set_option('display.max_columns', None)

c:\Users\garre\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [75]:
faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [76]:
fault_codes

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,Y,9996,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7120,Y,9997,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7121,Y,9998,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7122,Y,9999,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN


In [77]:
diagnostic_data

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
...,...,...,...,...
12821621,12864020,EngineCoolantTemperature,181.4,1248457
12821622,12864021,ParkingBrake,False,1248457
12821623,12864022,SwitchedBatteryVoltage,14.1,1248457
12821624,12864023,DistanceLtd,28606.65625,1248457


clean data

In [78]:
# Define the service locations
service_locations = [(36.0666667, -86.4347222), (35.5883333, -86.4438888), (36.1950, -83.174722)]

# Filter out faults occurring within 1 mile of service locations
for loc in service_locations:
    lat_mile = 2/69 # Convert 1 mile to degrees of latitude
    lon_mile = 1/(69*np.cos(np.radians(loc[0]))) * 2 # Convert 1 mile to degrees of longitude at the given latitude
    faults = faults[(np.abs(faults['Latitude'] - loc[0]) > lat_mile) | (np.abs(faults['Longitude'] - loc[1]) > lon_mile)]

# Filter out faults where the EquipmentID has more than 5 characters
faults = faults[faults['EquipmentID'].str.len() <= 5]
faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [79]:
wide_diag = diagnostic_data.pivot(index='FaultId', columns='Name', values='Value')
wide_diag

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,,,,
1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,1310.25,10722.7,96.4,58979.184415546,7.647805,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,1340.75,9326.75,100,65080.10587046,8.995086,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96


In [80]:
fault_diag = pd.merge(left = faults,
         right = wide_diag,
         left_on = 'RecordID',
         right_on = 'FaultId'
        )
fault_diag

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051207,1248454,123904424,2020-03-06 14:00:26.000,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1051208,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,1310.25,10722.7,96.4,58979.184415546,7.647805,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1051209,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,1340.75,9326.75,100,65080.10587046,8.995086,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1051210,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,863.25,586.75,23.6,4042.49282573,0,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [81]:
equipment = fault_diag[fault_diag['spn'] == 5246]['EquipmentID'].unique()

In [82]:
fault_diag_filtered = fault_diag[fault_diag['EquipmentID'].isin(equipment)]
fault_diag_filtered

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67
9,10,990446,2015-02-21 11:41:55.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1417,105438630,33.039953,-96.182592,2015-02-21 11:41:51.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
10,11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000,NaN,14.1375,False,64.6226,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69
13,14,990470,2015-02-21 11:43:40.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1582,105410980,40.161944,-80.161018,2015-02-21 11:43:36.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
15,16,990495,2015-02-21 11:45:06.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1601,105430420,38.272037,-78.937129,2015-02-21 11:45:02.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051183,1248429,123886552,2020-03-06 12:11:54.000,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,False,126,NaN,1853,105376821,26.079814,-80.123518,2020-03-06 12:11:49.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65535,NaN,NaN,NaN,NaN,NaN,NaN
1051185,1248431,123891846,2020-03-06 12:20:36.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,NaN,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17.000,0,14.645,False,66.48672,443434.9,186.8,12,22.62,203.7875,601,10755.85,54,62063.92146675,0.7132668,32,True,143.6,1279,True,NaN,0,NaN,100,0.58
1051186,1248432,123891847,2020-03-06 12:20:36.000,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,True,126,NaN,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17.000,0,14.645,False,66.48672,443434.9,186.8,12,22.62,203.7875,601,10755.85,54,62063.92146675,0.7132668,32,True,143.6,1279,True,NaN,0,NaN,100,0.58
1051192,1248438,123893761,2020-03-06 12:51:53.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,False,126,NaN,1853,105376821,26.078888,-80.122361,2020-03-06 12:51:49.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65535,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
fault_diag_filtered = fault_diag_filtered[fault_diag_filtered['active'] == True]
fault_diag_filtered

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67
10,11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000,NaN,14.1375,False,64.6226,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69
16,17,990504,2015-02-21 11:45:14.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1601,105430420,38.279629,-78.935509,2015-02-21 11:45:51.000,37.2,14.1375,False,66.48672,268575.9,183.2,26,33.64,217.2875,1035.5,5004.75,66,39450.13338542,2.166218,NaN,True,60.8,17407,NaN,NaN,36.38905,3276.75,NaN,1.16
23,29,990653,2015-02-21 11:53:52.000,Abnormal Update Rate Tire Location,NaN,unknown,unknown,unknown,unknown,49,929,9,True,126,NaN,1605,105415080,37.521898,-85.881759,2015-02-21 11:54:29.000,0,14.21,False,66.48672,231667.7,168.8,13,26.1,204.4063,599.875,4378.65,NaN,32180.911030536,0.594389,32,True,96.8,1279,True,NaN,0,3276.75,0,0.29
28,35,990771,2015-02-21 12:01:10.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00187677*082113134117*07700053*I0*BBZ*,79619763,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1585,105443412,38.904675,-85.823657,2015-02-21 12:01:53.000,56.8,14.355,False,66.48672,319674.2,186.8,52,34.8,220.2688,1386.5,5791.95,29.2,48822.165274224,12.0727,NaN,True,82.4,1023,NaN,NaN,65.20514,3276.75,NaN,19.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051154,1248395,123872855,2020-03-06 10:46:24.000,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,True,126,NaN,1853,105376821,25.994953,-80.182175,2020-03-06 10:57:23.000,0,14.645,False,66.48672,443424,185,12,24.94,193.8313,600.625,10755.3,68.8,62061.94017636,0.7264754,32,True,136.4,1279,True,NaN,0,NaN,100,0.29
1051179,1248425,123884064,2020-03-06 12:00:41.000,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,True,126,NaN,1853,105376821,26.080324,-80.124166,2020-03-06 12:01:17.000,0,14.645,False,66.48672,443434.3,185,17,22.04,208.7937,599.625,10755.7,50.4,62063.657294698,0.7528927,32,True,141.8,1279,True,NaN,0,NaN,100,0.58
1051180,1248426,123884065,2020-03-06 12:00:41.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,NaN,1853,105376821,26.080324,-80.124166,2020-03-06 12:01:17.000,0,14.645,False,66.48672,443434.3,185,17,22.04,208.7937,599.625,10755.7,50.4,62063.657294698,0.7528927,32,True,141.8,1279,True,NaN,0,NaN,100,0.58
1051185,1248431,123891846,2020-03-06 12:20:36.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,NaN,1853,105376821,26.078518,-80.122268,2020-03-06 12:41:17.000,0,14.645,False,66.48672,443434.9,186.8,12,22.62,203.7875,601,10755.85,54,62063.92

drop irrelevant columns from dataset

In [84]:
drop_columns = ['ecuSoftwareVersion', 'ecuSerialNumber', 'actionDescription', 'faultValue', 'CruiseControlActive', 'CruiseControlSetSpeed', 'ParkingBrake', 'ServiceDistance', 'SwitchedBatteryVoltage', 'MCTNumber', 'Throttle']
fault_diag_filtered = fault_diag_filtered.drop(drop_columns, axis=1)
fault_diag_filtered = fault_diag_filtered.replace(',', '.', regex=True)
fault_diag_filtered

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,TurboBoostPressure
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,0,111,17,True,1,1417,33.043564,-96.179722,2015-02-21 11:40:59.000,48,14.4275,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,13.6022,6.67
10,11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,0,111,17,True,1,1582,40.158055,-80.151018,2015-02-21 11:42:55.000,NaN,14.1375,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,64.82649,17.69
16,17,990504,2015-02-21 11:45:14.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,0,111,17,True,1,1601,38.279629,-78.935509,2015-02-21 11:45:51.000,37.2,14.1375,268575.9,183.2,26,33.64,217.2875,1035.5,5004.75,66,39450.13338542,2.166218,NaN,True,60.8,17407,36.38905,1.16
23,29,990653,2015-02-21 11:53:52.000,Abnormal Update Rate Tire Location,unknown,unknown,49,929,9,True,126,1605,37.521898,-85.881759,2015-02-21 11:54:29.000,0,14.21,231667.7,168.8,13,26.1,204.4063,599.875,4378.65,NaN,32180.911030536,0.594389,32,True,96.8,1279,0,0.29
28,35,990771,2015-02-21 12:01:10.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,0,111,17,True,1,1585,38.904675,-85.823657,2015-02-21 12:01:53.000,56.8,14.355,319674.2,186.8,52,34.8,220.2688,1386.5,5791.95,29.2,48822.165274224,12.0727,NaN,True,82.4,1023,65.20514,19.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051154,1248395,123872855,2020-03-06 10:46:24.000,High Voltage (Left Fuel Level Sensor),CECU3B-NAMUX4,PACCR,49,829,3,True,126,1853,25.994953,-80.182175,2020-03-06 10:57:23.000,0,14.645,443424,185,12,24.94,193.8313,600.625,10755.3,68.8,62061.94017636,0.7264754,32,True,136.4,1279,0,0.29
1051179,1248425,123884064,2020-03-06 12:00:41.000,High Voltage (Left Fuel Level Sensor),CECU3B-NAMUX4,PACCR,49,829,3,True,126,1853,26.080324,-80.124166,2020-03-06 12:01:17.000,0,14.645,443434.3,185,17,22.04,208.7937,599.625,10755.7,50.4,62063.657294698,0.7528927,32,True,141.8,1279,0,0.58
1051180,1248426,123884065,2020-03-06 12:00:41.000,High Voltage (Fuel Level),CECU3B-NAMUX4,PACCR,49,96,3,True,126,1853,26.080324,-80.124166,2020-03-06 12:01:17.000,0,14.645,443434.3,185,17,22.04,208.7937,599.625,10755.7,50.4,62063.657294698,0.7528927,32,True,141.8,1279,0,0.58
1051185,1248431,123891846,2020-03-06 12:20:36.000,High Voltage (Fuel Level),CECU3B-NAMUX4,PACCR,49,96,3,True,126,1853,26.078518,-80.122268,2020-03-06 12:41:17.000,0,14.645,443434.9,186.8,12,22.62,203.7875,601,10755.85,54,62063.92146675,0.7132668,32,True,143.6,1279,0,0.58


In [85]:
 # create a new DataFrame with only the numeric columns
dropped_columns = ['EventTimeStamp', 'eventDescription', 'ecuModel', 'ecuMake', 'LocationTimeStamp', 'IgnStatus']
numeric_data = fault_diag_filtered.drop(dropped_columns, axis=1)

# impute missing values
imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_data = pd.DataFrame(imputer.fit_transform(numeric_data), columns=numeric_data.columns)

# select only the columns you want to keep from the original DataFrame
keep_columns = ['RecordID', 'EventTimeStamp', 'eventDescription', 'ecuModel', 'ecuMake', 'LocationTimeStamp', 'IgnStatus']

# merge the selected columns with the imputed data
imputed_df = pd.merge(fault_diag_filtered[keep_columns], imputed_data, on='RecordID', how='left')
imputed_df


,RecordID,EventTimeStamp,eventDescription,ecuModel,ecuMake,LocationTimeStamp,IgnStatus,ESS_Id,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,AcceleratorPedal,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,Speed,TurboBoostPressure
0,6,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,2015-02-21 11:40:59.000,True,990431.0,0.0,111.0,17.0,1.0,1.0,1417.0,33.043564,-96.179722,48.000000,14.4275,470381.4,181.4,30.0,38.28,196.5313,1514.500,9480.00,44.000000,70349.809964,4.583399,73.209105,111.2,1023.0,13.60220,6.67
1,11,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,2015-02-21 11:42:55.000,True,990459.0,0.0,111.0,17.0,1.0,1.0,1582.0,40.158055,-80.151018,73.126818,14.1375,257885.9,181.4,80.0,34.22,218.9750,1382.375,5085.35,42.800000,37724.957800,13.723780,54.731737,78.8,1023.0,64.82649,17.69
2,17,2015-02-21 11:45:14.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,2015-02-21 11:45:51.000,True,990504.0,0.0,111.0,17.0,1.0,1.0,1601.0,38.279629,-78.935509,37.200000,14.1375,268575.9,183.2,26.0,33.64,217.2875,1035.500,5004.75,66.000000,39450.133385,2.166218,55.987690,60.8,17407.0,36.38905,1.16
3,29,2015-02-21 11:53:52.000,Abnormal Update Rate Tire Location,unknown,unknown,2015-02-21 11:54:29.000,True,990653.0,49.0,929.0,9.0,1.0,126.0,1605.0,37.521898,-85.881759,0.000000,14.2100,231667.7,168.8,13.0,26.10,204.4063,599.875,4378.65,57.208972,32180.911031,0.594389,32.000000,96.8,1279.0,0.00000,0.29
4,35,2015-02-21 12:01:10.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,2015-02-21 12:01:53.000,True,990771.0,0.0,111.0,17.0,1.0,1.0,1585.0,38.904675,-85.823657,56.800000,14.3550,319674.2,186.8,52.0,34.80,220.2688,1386.500,5791.95,29.200000,48822.165274,12.072700,76.121905,82.4,1023.0,65.20514,19.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175776,1248395,2020-03-06 10:46:24.000,High Voltage (Left Fuel Level Sensor),CECU3B-NAMUX4,PACCR,2020-03-06 10:57:23.000,True,123872855.0,49.0,829.0,3.0,1.0,126.0,1853.0,25.994953,-80.182175,0.000000,14.6450,443424.0,185.0,12.0,24.94,193.8313,600.625,10755.30,68.800000,62061.940176,0.726475,32.000000,136.4,1279.0,0.00000,0.29
175777,1248425,2020-03-06 12:00:41.000,High Voltage (Left Fuel Level Sensor),CECU3B-NAMUX4,PACCR,2020-03-06 12:01:17.000,True,123884064.0,49.0,829.0,3.0,1.0,126.0,1853.0,26.080324,-80.124166,0.000000,14.6450,443434.3,185.0,17.0,22.04,208.7937,599.625,10755.70,50.400000,62063.657295,0.752893,32.000000,141.8,1279.0,0.00000,0.58
175778,1248426,2020-03-06 12:00:41.000,High Voltage (Fuel Level),CECU3B-NAMUX4,PACCR,2020-03-06 12:01:17.000,True,123884065.0,49.0,96.0,3.0,1.0,126.0,1853.0,26.080324,-80.124166,0.000000,14.6450,443434.3,185.0,17.0,22.04,208.7937,599.625,10755.70,50.400000,62063.657295,0.752893,32.000000,141.8,1279.0,0.00000,0.58
175779,1248431,2020-03-06 12:20:36.000,High Voltage (Fuel Level),CECU3B-NAMUX4,PACCR,2020-03-06 12:41:17.000,True,123891846.0,49.0,96.0,3.0,1.0,126.0,1853.0,26.078518,-80.122268,0.000000,14.6450,443434.9,186.8,12.0,22.62,203.7875,601.000,10755.85,54.000000,62063.921467,0.713267,32.000000,143.6,1279.0,0.00000,0.58


In [86]:
# Count the number of NaN values in each column
num_nan = imputed_df.isna().sum()

print(num_nan)


RecordID                        0
EventTimeStamp                  0
eventDescription             6781
ecuModel                     4940
ecuMake                      4940
LocationTimeStamp               0
IgnStatus                       0
ESS_Id                          0
ecuSource                       0
spn                             0
fmi                             0
active                          0
activeTransitionCount           0
EquipmentID                     0
Latitude                        0
Longitude                       0
AcceleratorPedal                0
BarometricPressure              0
DistanceLtd                     0
EngineCoolantTemperature        0
EngineLoad                      0
EngineOilPressure               0
EngineOilTemperature            0
EngineRpm                       0
EngineTimeLtd                   0
FuelLevel                       0
FuelLtd                         0
FuelRate                        0
FuelTemperature                 0
IntakeManifold

In [87]:
# Count the number of NaN values in each column
num_nan = fault_diag_filtered.isna().sum()

print(num_nan)


RecordID                         0
ESS_Id                           0
EventTimeStamp                   0
eventDescription              6781
ecuModel                      4940
ecuMake                       4940
ecuSource                        0
spn                              0
fmi                              0
active                           0
activeTransitionCount            0
EquipmentID                      0
Latitude                         0
Longitude                        0
LocationTimeStamp                0
AcceleratorPedal             33195
BarometricPressure            5877
DistanceLtd                   5919
EngineCoolantTemperature      5946
EngineLoad                    5935
EngineOilPressure             5817
EngineOilTemperature          6269
EngineRpm                     5727
EngineTimeLtd                 7500
FuelLevel                    18297
FuelLtd                       5915
FuelRate                      6004
FuelTemperature              92578
IgnStatus           

In [88]:
fault_diag_filtered.head(8)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,TurboBoostPressure
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,0,111,17,True,1,1417,33.043564,-96.179722,2015-02-21 11:40:59.000,48,14.4275,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,13.6022,6.67
10,11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,0,111,17,True,1,1582,40.158055,-80.151018,2015-02-21 11:42:55.000,NaN,14.1375,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,64.82649,17.69
16,17,990504,2015-02-21 11:45:14.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,0,111,17,True,1,1601,38.279629,-78.935509,2015-02-21 11:45:51.000,37.2,14.1375,268575.9,183.2,26,33.64,217.2875,1035.5,5004.75,66,39450.13338542,2.166218,NaN,True,60.8,17407,36.38905,1.16
23,29,990653,2015-02-21 11:53:52.000,Abnormal Update Rate Tire Location,unknown,unknown,49,929,9,True,126,1605,37.521898,-85.881759,2015-02-21 11:54:29.000,0,14.21,231667.7,168.8,13,26.1,204.4063,599.875,4378.65,NaN,32180.911030536,0.594389,32,True,96.8,1279,0,0.29
28,35,990771,2015-02-21 12:01:10.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,0,111,17,True,1,1585,38.904675,-85.823657,2015-02-21 12:01:53.000,56.8,14.355,319674.2,186.8,52,34.8,220.2688,1386.5,5791.95,29.2,48822.165274224,12.0727,NaN,True,82.4,1023,65.20514,19.43
37,49,990990,2015-02-21 12:14:51.000,Incorrect Data Brake Signal Sensor 2,unknown,unknown,11,1068,2,True,127,1437,35.874166,-84.235462,2015-02-21 12:15:27.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN
42,56,991057,2015-02-21 12:18:38.000,Special Instructions System Diagnostic Code #1,unknown,unknown,11,611,14,True,127,305,33.673750,-84.340324,2015-02-21 12:19:14.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN
43,57,991058,2015-02-21 12:18:38.000,Special Instructions Wheel Sensor ABS Axle 2 Left,unknown,unknown,11,791,14,True,127,305,33.673750,-84.340324,2015-02-21 12:19:14.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN


In [89]:
imputed_df.head(8)

,RecordID,EventTimeStamp,eventDescription,ecuModel,ecuMake,LocationTimeStamp,IgnStatus,ESS_Id,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,AcceleratorPedal,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,Speed,TurboBoostPressure
0,6,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,2015-02-21 11:40:59.000,True,990431.0,0.0,111.0,17.0,1.0,1.0,1417.0,33.043564,-96.179722,48.000000,14.427500,470381.400000,181.400000,30.000000,38.280000,196.531300,1514.500000,9480.000000,44.000000,70349.809964,4.583399,73.209105,111.200000,1023.0,13.602200,6.670000
1,11,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,2015-02-21 11:42:55.000,True,990459.0,0.0,111.0,17.0,1.0,1.0,1582.0,40.158055,-80.151018,73.126818,14.137500,257885.900000,181.400000,80.000000,34.220000,218.975000,1382.375000,5085.350000,42.800000,37724.957800,13.723780,54.731737,78.800000,1023.0,64.826490,17.690000
2,17,2015-02-21 11:45:14.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,2015-02-21 11:45:51.000,True,990504.0,0.0,111.0,17.0,1.0,1.0,1601.0,38.279629,-78.935509,37.200000,14.137500,268575.900000,183.200000,26.000000,33.640000,217.287500,1035.500000,5004.750000,66.000000,39450.133385,2.166218,55.987690,60.800000,17407.0,36.389050,1.160000
3,29,2015-02-21 11:53:52.000,Abnormal Update Rate Tire Location,unknown,unknown,2015-02-21 11:54:29.000,True,990653.0,49.0,929.0,9.0,1.0,126.0,1605.0,37.521898,-85.881759,0.000000,14.210000,231667.700000,168.800000,13.000000,26.100000,204.406300,599.875000,4378.650000,57.208972,32180.911031,0.594389,32.000000,96.800000,1279.0,0.000000,0.290000
4,35,2015-02-21 12:01:10.000,Low (Severity Low) Engine Coolant Level,6X1u10D1500000000,CMMNS,2015-02-21 12:01:53.000,True,990771.0,0.0,111.0,17.0,1.0,1.0,1585.0,38.904675,-85.823657,56.800000,14.355000,319674.200000,186.800000,52.000000,34.800000,220.268800,1386.500000,5791.950000,29.200000,48822.165274,12.072700,76.121905,82.400000,1023.0,65.205140,19.430000
5,49,2015-02-21 12:14:51.000,Incorrect Data Brake Signal Sensor 2,unknown,unknown,2015-02-21 12:15:27.000,True,990990.0,11.0,1068.0,2.0,1.0,127.0,1437.0,35.874166,-84.235462,29.574331,14.363972,293438.493268,176.160389,35.255694,36.702629,199.178498,1067.755770,4570.753140,61.480000,36281.613393,5.099437,72.032622,111.637001,1279.0,28.355514,6.986988
6,56,2015-02-21 12:18:38.000,Special Instructions System Diagnostic Code #1,unknown,unknown,2015-02-21 12:19:14.000,True,991057.0,11.0,611.0,14.0,1.0,127.0,305.0,33.673750,-84.340324,25.715504,14.488784,344266.650086,170.734446,34.071753,41.679588,183.116481,925.897392,7277.161070,72.655247,45564.468783,5.003865,94.687726,115.970084,1279.0,17.655025,6.833915
7,57,2015-02-21 12:18:38.000,Special Instructions Wheel Sensor ABS Axle 2 Left,unknown,unknown,2015-02-21 12:19:14.000,True,991058.0,11.0,791.0,14.0,1.0,127.0,305.0,33.673750,-84.340324,25.713721,14.488788,344264.350973,170.736433,34.072898,41.679004,183.117811,925.874611,7277.073949,72.657833,45562.872035,5.003870,94.695629,115.969156,1279.0,17.652753,6.833657


Rolling Windows

In [92]:
# convert 'EventTimeStamp' column to datetime object
imputed_df['EventTimeStamp'] = pd.to_datetime(imputed_df['EventTimeStamp'])

# set the window size to 24 hours
window_size = '24'

# create a rolling window object for the 'EventTimeStamp' column
event_rolling = imputed_df['EventTimeStamp'].rolling(window=window_size)

# calculate the count of events in the previous 24 hours for each row
imputed_df['Events_Last_24H'] = event_rolling.count()


ValueError: window must be an integer 0 or greater

In [71]:
# set the window size to 3
window_size = 3

# create a rolling window object for the 'EventTimeStamp' column
event_rolling = imputed_df['EventTimeStamp'].rolling(window=window_size)

# create a rolling window object for the 'LocationTimeStamp' column
location_rolling = imputed_df['LocationTimeStamp'].rolling(window=window_size)

# calculate the rolling mean for each column
imputed_df['EventTimeStamp_RollingMean'] = event_rolling.mean()
imputed_df['LocationTimeStamp_RollingMean'] = location_rolling.mean()


DataError: No numeric types to aggregate

In [ ]:
derate_faults = faults[faults['spn'].isin([5246, 1569])]
derate_faults


In [ ]:
fault_diag_filtered['FuelTemperature'].value_counts()


In [ ]:
full_derate = derate_faults[derate_faults['spn'].isin([5246])]
full_derate

In [ ]:
full_derate['FuelTemperature'].value_counts()